# ✏️ 형태소 분석

In [ ]:
import pymongo
import pandas as pd

import MeCab
import neologdn
from nltk.corpus import stopwords

from collections import Counter

import googletrans


# 분석할 단어만 추출
def is_content_word(feature):
  
  # 포함 품사: 명사, 동사, 형용사, 부사
  CONTENT_WORD_POS = ("名詞", "動詞", "形容詞", "副詞")
  # 제외 품사: 접미사, 비자립, 대명사, 미지어
  IGNORE = ("接尾", "非自立", "代名詞", "未知語")

  return feature.startswith(CONTENT_WORD_POS) and all(f not in IGNORE for f in feature.split(",")[:7])


# 형태소 분석
def morphological_analysis(data):
  
  text = neologdn.normalize(data[2], repeat=2)
  text = "".join([i for i in text if i.isalpha() or i.isspace()])
  
  # 형태소 분석
  mecab = MeCab.Tagger("-O wakati -F%m\\t --unk-feature 未知語")
  node = mecab.parseToNode(text)
  
  # 형태소 분석 결과 저장 리스트
  # 불용어 설정
  stop_words = stopwords.words("japanese")
  
  word_list = []
  
  while node:
    if is_content_word(node.feature):
      
      # 단어장에 넣을 단어 설정
      lemma = node.feature.split(",")[7] if len(node.feature.split(",")) > 7 and node.feature.split(",")[7] != "*" else node.surface
            
      # 불용어 제외하고 word_list에 저장
      if lemma not in stop_words:
        word_list.append(lemma.split("-")[0])
      
    node = node.next
  
  dbcol_subtitle.update_one({"filename": data["filename"]}, {"$set": {"words": word_list}})

    
# 빈도수 계산  
def word_frequency(data):
  
  count = Counter(data["words"])
  
  # 단어와 빈도수 딕셔너리 저장 리스트
  frequency = dict()
  
  for word, cnt in count.most_common():
    frequency[word] = cnt
  
  dbcol_subtitle.update_one({"filename": data["filename"]}, {"$set": {"frequency": frequency}})
  
# MongoDB와 연결
conn = pymongo.MongoClient("localhost", 27017)

# DB 선택
db = conn.soaeng

# Collection 선택
dbcol_subtitle = db.subtitle

# subtitle 데이터를 프레임으로 설정
df = pd.DataFrame(list(dbcol_subtitle.find({})))

# 형태소 분석
df.apply(morphological_analysis, axis = 1)

# 빈도수 계산
df.apply(word_frequency, axis=1)

print("저장 완료")

# ✏️ 빈도수 통합

In [ ]:
import pymongo
import pandas as pd

def save_words(data):
  word_list = data["frequency"]
  for word, cnt in word_list.items():
    if dbcol_word.find_one({"word": word}) != None:
      count = dbcol_word.find_one({"word": word})["frequency"]
    else:
      count = 0
    cnt += count
    dbcol_word.update_one({"word": word}, {"$set": {"frequency": cnt}}, upsert = True)
  dbcol_subtitle.update_one({"filename": data["filename"]}, {"$set": {"check": 1}})

# MongoDB와 연결
conn = pymongo.MongoClient("localhost", 27017)

# DB 선택
db = conn.soaeng

# Collection 선택
dbcol_subtitle = db.subtitle
dbcol_word = db.word

# subtitle 데이터를 프레임으로 설정
df = pd.DataFrame(list(dbcol_subtitle.find( {"$or": [{"check": {"$exists": False}}, {"check": {"$eq": 0}}]})))

print(df.apply(save_words, axis = 1))

print("단어 저장 완료")

# ✏️ 한국어 번역

### 🔷 googletrans

In [35]:
from googletrans import Translator

def translation_using_googletrans(data):
    
  word = data["word"]

  trans = Translator()
  trans.raise_Exception = True

  # 일-한 단어 번역
  o = trans.translate(text=word, src = "ja", dest = "ja") # for 발음 표기
  t = trans.translate(text=word, src = "ja", dest = "ko")
  
  pronunciation = o.pronunciation
  korean = t.text

  dbcol_word.update_one({"word": word}, {"$set": {"korean": korean, "pronunciation": pronunciation}})

# MongoDB와 연결
conn = pymongo.MongoClient("localhost", 27017)

# DB 선택
db = conn.soaeng

# Collection 선택
dbcol_word = db.word

# subtitle 데이터를 프레임으로 설정
df = pd.DataFrame(list(dbcol_word.find( {"$or": [{"korean": {"$exists": False}}, {"pronunciation": {"$exists": False}}]}).sort("frequency", -1).limit(1000)))
# print(df)
df.apply(translation_using_googletrans, axis = 1)

print("단어 번역 완료 - google")

단어 번역 완료 - google


### 🔷 papago

In [4]:
import dotenv
import os
import pymongo
import pandas as pd
import requests

dotenv.load_dotenv(dotenv.find_dotenv())
client_id = os.environ["PAPAGO_ID"] # 개발자센터에서 발급받은 Client ID 값
client_secret = os.environ["PAPAGO_SECRET"] # 개발자센터에서 발급받은 Client Secret 값

url = "https://openapi.naver.com/v1/papago/n2mt"

def translation_using_papago(data):
    
    word = data["word"]
    data = {"text" : word, "source" : "ja", "target": "ko"}
#     data = "source=" + src + "&target=" +tgt+ "&text=" + word
        
    header = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data= data)
    rescode = response.status_code

    if(rescode==200):
        t_data = response.json()
        result = response.json()["message"]["result"]["translatedText"]
        dbcol_word.update_one({"word": word}, {"$set": {"papago": result}})
    else:
        print("Error Code:" , rescode)
        return 0
    
# MongoDB와 연결
conn = pymongo.MongoClient("localhost", 27017)

# DB 선택
db = conn.soaeng

# Collection 선택
dbcol_word = db.word

# subtitle 데이터를 프레임으로 설정
df = pd.DataFrame(list(dbcol_word.find( {"$or": [{"papago": {"$exists": False}}]}).sort("frequency", -1).limit(1000)))
# print(df)
df.apply(translation_using_papago, axis = 1)

print("단어 번역 완료 - naver")


단어 번역 완료 - naver
